In [18]:
import pandas as pd
from pathlib import Path
import re
import string
import nltk

from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

tweets_df = pd.read_csv(
    Path('twitter_training.csv')
)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\magey\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
tweets_df = tweets_df.rename({'Positive':'sentiment', 'im getting on borderlands and i will murder you all ,':'tweets'}, axis='columns').drop(columns=['2401','Borderlands'])
tweets_df.head(10)

,sentiment,tweets
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
5,Positive,So I spent a few hours making something for fu...
6,Positive,So I spent a couple of hours doing something f...
7,Positive,So I spent a few hours doing something for fun...
8,Positive,So I spent a few hours making something for fu...
9,Positive,2010 So I spent a few hours making something f...


In [20]:
tweets_df = tweets_df.query("sentiment != 'Irrelevant'")
tweets_df.head()

,sentiment,tweets
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [21]:
sentiment_ordering = ['Negative', 'Neutral', 'Positive']
tweets_df['sentiment'] = tweets_df['sentiment'].apply(lambda x: sentiment_ordering.index(x))
tweets_df.head()

,sentiment,tweets
0,2,I am coming to the borders and I will kill you...
1,2,im getting on borderlands and i will kill you ...
2,2,im coming on borderlands and i will murder you...
3,2,im getting on borderlands 2 and i will murder ...
4,2,im getting into borderlands and i can murder y...


In [22]:
tweets_df = tweets_df.dropna()
sia = SentimentIntensityAnalyzer()
tweets_df['scores'] = tweets_df['tweets'].apply(lambda x: sia.polarity_scores(x)['compound'])
tweets_df.head()

,sentiment,tweets,scores
0,2,I am coming to the borders and I will kill you...,-0.6908
1,2,im getting on borderlands and i will kill you ...,-0.6908
2,2,im coming on borderlands and i will murder you...,-0.6908
3,2,im getting on borderlands 2 and i will murder ...,-0.6908
4,2,im getting into borderlands and i can murder y...,-0.6908


In [35]:
tweets_df['est_sentiment'] = tweets_df['scores'].apply(lambda x: 'positive' if x > 0.25 else 'negative' if x < -0.25 else 'neutral')
tweets_df.head(30)

,sentiment,tweets,scores,est_sentiment
0,2,I am coming to the borders and I will kill you...,-0.6908,negative
1,2,im getting on borderlands and i will kill you ...,-0.6908,negative
2,2,im coming on borderlands and i will murder you...,-0.6908,negative
3,2,im getting on borderlands 2 and i will murder ...,-0.6908,negative
4,2,im getting into borderlands and i can murder y...,-0.6908,negative
5,2,So I spent a few hours making something for fu...,0.9431,positive
6,2,So I spent a couple of hours doing something f...,0.9273,positive
7,2,So I spent a few hours doing something for fun...,0.4206,positive
8,2,So I spent a few hours making something for fu...,0.9431,positive
9,2,2010 So I spent a few hours making something f...,0.9431,positive


In [62]:
from sklearn.feature_extraction.text import CountVectorizer #Convert text data to numerical vectors
from sklearn.model_selection import train_test_split #Split data into train and test
from sklearn.naive_bayes import MultinomialNB #model being used


In [25]:
vectorizer = CountVectorizer() #Initializing vectorizer 
vectors = vectorizer.fit_transform(tweets_df["tweets"]) #Converting words to vectors

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(vectors, tweets_df["est_sentiment"])

In [64]:
model = MultinomialNB()
model.fit(X_train, Y_train)

MultinomialNB()

In [65]:
y_pred = model.predict(X_test)

In [49]:
predictions = pd.DataFrame({'True': Y_test, 'Prediction': y_pred})
predictions.head(30)

,True,Prediction
32118,negative,negative
21193,negative,negative
35485,negative,positive
68128,positive,positive
17466,neutral,neutral
47894,negative,negative
54042,neutral,neutral
48876,positive,positive
1065,negative,negative
66121,negative,negative


In [66]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy = ", accuracy)

Accuracy =  0.7489528795811519
